In [2]:
# import pandas as pd
# answer_df = pd.read_csv("../../data2/answer_df_mistral.csv")
# answer_df.iloc[0:500].to_csv("../../data2/answer_df_mistral1.csv", index=False)
# answer_df.iloc[500:1000].to_csv("../../data2/answer_df_mistral2.csv", index=False)
# answer_df.iloc[1000:].to_csv("../../data2/answer_df_mistral3.csv", index=False)


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch


print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             torch_dtype=torch.bfloat16, 
                                             device_map=device)



True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
piece_id = 1
answer_df_path = "../../data2/answer_df_mistral"+str(piece_id)+".csv"

In [4]:
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data2/fea_df.csv")

if not 'answer_string' in answer_df.columns:
    answer_df['answer_string'] = np.nan
    
print(answer_df.shape)
print(answer_df.iloc[(answer_df.shape[0]-10):answer_df.shape[0],[0,1,2]])
query_df

(500, 3)
     sm_id                                         text_w_eos  answer_string
490   1892  telling me to grow up and snap out of it.     ...            NaN
491   1764                                       about weight            NaN
492   2053  your weight is fine... comments on appearance,...            NaN
493   2192  primary care doctor congratulated me on losing...            NaN
494   1650                                     weight related            NaN
495   4779              your not skinny, do you have a phych,            NaN
496    787  i weigh too much to have an eating disorder  i...            NaN
497   2563  an old psychiatrist said i was okay (when i wa...            NaN
498    508  after disclosing my restrictive behaviors to m...            NaN
499   3670        weight-loss encouragement, lack of support.            NaN


,fea,description
0,weightbasedhealth,Judgments about health based on weight or appe...
1,weightstigma,"Negative attitudes, discrimination, or prejudi..."
2,weightbasedhealthcare,Healthcare decisions influenced by weight. Not...
3,encourageweightloss,Encouragement of weight loss
4,dietpromotion,Advocacy for diets or restrictive eating
5,thinness,Reassurance of thinness. Maintaining thinness
6,notlowweightenough,"Not sick enough, or not thin enought, or not l..."
7,weightblame,Weight blamed for health issues or concerns
8,weightvalue,Weight tied to personality characteristics


In [6]:
# # in replace of system message
# def get_topic_label(label: str, relatedphrases: str):
#     """
#     Answer whether any topic exists in a paragraph. Only return answer.
    
#     Args:
#         label: whether a topic exists in the given paragraph, (yes/no).
#         relatedphrases: if yes, provide the phrases in the paragraph that are related to the topic. (related phrases: ...)
#     """
#     pass
# in replace of system message
system_message = """
You are an AI assistant designed to answer questions.
Use the exact answer format asked.
"""

def get_topic_label():
    """
    Answer questions.
    Restrict to the exact answer format.
    Provide related phrases from the paragraph, only if the answer is yes.
    """
    pass
tools = [get_topic_label]

def format_user_message(text):
    question_content = "The provided parapraph is eating disorders patients' experiences with physicians. Weight stigma is indicated by weight-or-appearance-related treatment or judgement in this context. Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format: Answer starts here:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: yes/no, related phrases if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = question_content + paragragh_content + answer_content
    #print(user_message)
    
    return user_message




In [7]:
from tqdm import tqdm

for k in tqdm(range(1000), desc="Processing batch"):
    
    batch_size = 10

    # Filter for rows where 'answer_string' is NaN
    unanswered_df = answer_df[answer_df['answer_string'].isna()]

    # Get the indices of these NaN entries in the original DataFrame
    indices_to_update = unanswered_df.index[:batch_size]

    # Prepare prompt content for the first 10 entries with NaN answer_string
    user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

    # Save the indices list if needed for later use
    indices_to_update_list = list(indices_to_update)


    if len(indices_to_update_list)>0:
        for i in range(len(user_messages)):
            failed = True
            failed_count = 0
            while failed and failed_count<10:
                conversation = [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_messages[i]}
                ]
                inputs = tokenizer.apply_chat_template(
                        conversation,
                        # tools=tools,
                        add_generation_prompt=False,
                        return_dict=True,
                        return_tensors="pt")
                inputs.to(model.device)
                outputs = model.generate(**inputs,
                                         do_sample=True,
                                         max_new_tokens=512,
                                         temperature=0.1)
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                del inputs
                if "Answer starts here:" in response:
                    answer = response.split("Answer starts here:")[-1]
                    # Use the original index from indices_to_update_list
                    answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer
                    # print(answer)
                    failed = False
                else:
                    failed = True
                    failed_count += 1
                    # Use the original index from indices_to_update_list
                    answer_df.loc[indices_to_update_list[i], 'answer_string'] = "Answer not found"

        print(answer_df.loc[indices_to_update_list, ['sm_id','answer_string']])
        answer_df.to_csv(answer_df_path, index=False)
    else:
         break
    

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   0%|          | 1/1000 [01:06<18:22:34, 66.22s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


   sm_id                                      answer_string
0   1690  \n  (1) weightbasedhealth: yes, the cardiologi...
1    724  \n  (1) weightbasedhealth: yes, related phrase...
2   2800  \n  (1) weightbasedhealth: yes, the person who...
3   1771  \n  (1) weightbasedhealth: yes, related phrase...
4   1975  \n  (1) weightbasedhealth: yes, related phrase...
5    126  \n  (1) weightbasedhealth: yes, related phrase...
6   1511  \n  (1) weightbasedhealth: yes, related phrase...
7   4091  \n  (1) weightbasedhealth: yes, related phrase...
8    371  \n  (1) weightbasedhealth: yes, the statement ...
9   4313  \n  (1) weightbasedhealth: yes, related phrase...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   0%|          | 2/1000 [02:15<18:46:50, 67.75s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


    sm_id                                      answer_string
10    430  \n  (1) weightbasedhealth: yes, related phrase...
11     70  \n  (1) weightbasedhealth: yes, related phrase...
12   2225  \n  (1) weightbasedhealth: yes, related phrase...
13    578  \n  (1) weightbasedhealth: yes, as the mention...
14    913  \n  (1) weightbasedhealth: yes, the doctor's s...
15   2367  \n  (1) weightbasedhealth: yes, related phrase...
16   4577  \n  (1) weightbasedhealth: yes, the statement ...
17   2536  \n  (1) weightbasedhealth: yes, related phrase...
18   1085  \n  (1) weightbasedhealth: yes, related phrase...
19   3916  \n  (1) weightbasedhealth: yes, related phrase...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   0%|          | 3/1000 [03:22<18:43:10, 67.59s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

     sm_id                                      answer_string
470    506  \n  (1) weightbasedhealth: yes, related phrase...
471   2526  \n  (1) weightbasedhealth: yes, related phrase...
472    788  \n  (1) weightbasedhealth: yes, related phrase...
473    269  \n  (1) weightbasedhealth: yes, related phrase...
474   1410  \n  (1) weightbasedhealth: yes, related phrase...
475   4265  \n  (1) weightbasedhealth: yes, related phrase...
476   4365  \n  (1) weightbasedhealth: yes, the nurse aske...
477    998  \n  (1) weightbasedhealth: yes, related phrase...
478    605  \n  (1) weightbasedhealth: yes, related phrase...
479   2362  \n  (1) weightbasedhealth: yes, related phrase...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   5%|▍         | 49/1000 [53:26<18:32:45, 70.21s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


     sm_id                                      answer_string
480   2352  \n  (1) weightbasedhealth: yes, related phrase...
481   2744  \n  (1) weightbasedhealth: yes, related phrase...
482   2617  \n  (1) weightbasedhealth: yes, related phrase...
483   2448  \n  (1) weightbasedhealth: yes, related phrase...
484   1748  \n  (1) weightbasedhealth: yes, the allergist ...
485   3517  \n  (1) weightbasedhealth: yes, related phrase...
486   1642  \n  (1) weightbasedhealth: yes, related phrase...
487   1769  \n  (1) weightbasedhealth: yes, the statement ...
488   3297  \n  (1) weightbasedhealth: yes, related phrase...
489   1595  \n  (1) weightbasedhealth: yes, related phrase...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   5%|▌         | 50/1000 [54:34<17:17:00, 65.49s/it]

     sm_id                                      answer_string
490   1892  \n  (1) weightbasedhealth: yes, related phrase...
491   1764  \n  (1) weightbasedhealth: yes, related phrase...
492   2053  \n  (1) weightbasedhealth: yes, the statement ...
493   2192  \n  (1) weightbasedhealth: yes, the doctor's c...
494   1650  \n  (1) weightbasedhealth: yes, related phrase...
495   4779  \n  (1) weightbasedhealth: yes, related phrase...
496    787  \n  (1) weightbasedhealth: yes, related phrase...
497   2563  \n  (1) weightbasedhealth: yes, the psychiatri...
498    508  \n  (1) weightbasedhealth: yes, the psychiatri...
499   3670  \n  (1) weightbasedhealth: yes, related phrase...


In [8]:
import pandas as pd

# File paths
file_paths = [
    "../../data2/answer_df_mistral1.csv",
    "../../data2/answer_df_mistral2.csv",
    "../../data2/answer_df_mistral3.csv"
]

# Read and concatenate dataframes
answer_df_list = [pd.read_csv(file_path) for file_path in file_paths]
answer_df = pd.concat(answer_df_list, ignore_index=True)

# Save the concatenated dataframe
output_path = "../../data2/answer_df_mistral.csv"
answer_df.to_csv(output_path, index=False)
